In [9]:
import pandas as pd
from datetime import datetime
import re

all_sheets = pd.read_excel("/Users/azmanizakary/Downloads/Integrated_Data_BAM.xlsx", sheet_name=None, index_col=0)

"""Iterate through all sheets
for sheet_name, df in all_sheets.items():
    print(f"Sheet Name: {sheet_name}")
    print(df.head())"""

'Iterate through all sheets\nfor sheet_name, df in all_sheets.items():\n    print(f"Sheet Name: {sheet_name}")\n    print(df.head())'

In [10]:
# All names of Dataframes by Sheet name

print(all_sheets.keys())

dict_keys(['unemployment_rate', 'extra_unemployment_rate', 'gov_total_expense_revenue', 'money_supply', 'bonds', 'cpi', 'economic_growth', 'import_export', 'uk_home_retail', 'seasonal_retail_sales', 'uk_building', 'uk_affordability_index', 'uk_retail_price', 'mortage', 'construction_cost_prices_sales', 'mortage_interest_rate', 'company_revenue', 'operating_expenses'])


In [11]:
all_sheets['extra_unemployment_rate']

,Unemployment Rate,Unemployment (Thousands),Female,Male
Date,,,,
Dec 2024,--,--,--,--
Nov 2024,4.4,1568,4.2,4.7
Oct 2024,4.3,1508,4,4.6
Sep 2024,4.3,1509,4.1,4.5
Aug 2024,4.1,1435,4,4.2
...,...,...,...,...
Sep 2018,4.1,1382,4,4.2
Aug 2018,4,1368,4,4.1
Jul 2018,4,1363,4,4


Clean data for columns that have all null values

In [12]:
# Drop unnamed columns, using this to clean data as tables such as Extra Unemployment Rate have been converted weirdly, giving extra null columns where there aren't any
def drop_unnamed_columns(df):
    return df.loc[:, ~df.columns.str.contains('^Unnamed')]

cleaned_columns = {name: drop_unnamed_columns(df) for name, df in all_sheets.items()}

cleaned_columns['extra_unemployment_rate']

,Unemployment Rate,Unemployment (Thousands),Female,Male
Date,,,,
Dec 2024,--,--,--,--
Nov 2024,4.4,1568,4.2,4.7
Oct 2024,4.3,1508,4,4.6
Sep 2024,4.3,1509,4.1,4.5
Aug 2024,4.1,1435,4,4.2
...,...,...,...,...
Sep 2018,4.1,1382,4,4.2
Aug 2018,4,1368,4,4.1
Jul 2018,4,1363,4,4


Dropping fully null rows

In [13]:
cleaned_data = {}

for sheet_name, df in cleaned_columns.items():
    print(f"\n📄 Cleaning Sheet: {sheet_name}")

    try:
        # 🚀 Drop rows where all values are null
        df_cleaned = df.dropna(how="all")

        # Store the cleaned DataFrame
        cleaned_data[sheet_name] = df_cleaned

        # Print number of rows dropped
        rows_dropped = len(df) - len(df_cleaned)
        print(f"✅ Dropped {rows_dropped} fully null rows from {sheet_name}")

    except Exception as e:
        print(f"❌ Error cleaning {sheet_name}: {e}")


📄 Cleaning Sheet: unemployment_rate
✅ Dropped 0 fully null rows from unemployment_rate

📄 Cleaning Sheet: extra_unemployment_rate
✅ Dropped 0 fully null rows from extra_unemployment_rate

📄 Cleaning Sheet: gov_total_expense_revenue
✅ Dropped 0 fully null rows from gov_total_expense_revenue

📄 Cleaning Sheet: money_supply
✅ Dropped 0 fully null rows from money_supply

📄 Cleaning Sheet: bonds
✅ Dropped 0 fully null rows from bonds

📄 Cleaning Sheet: cpi
✅ Dropped 0 fully null rows from cpi

📄 Cleaning Sheet: economic_growth
✅ Dropped 0 fully null rows from economic_growth

📄 Cleaning Sheet: import_export
✅ Dropped 0 fully null rows from import_export

📄 Cleaning Sheet: uk_home_retail
✅ Dropped 0 fully null rows from uk_home_retail

📄 Cleaning Sheet: seasonal_retail_sales
✅ Dropped 0 fully null rows from seasonal_retail_sales

📄 Cleaning Sheet: uk_building
✅ Dropped 0 fully null rows from uk_building

📄 Cleaning Sheet: uk_affordability_index
✅ Dropped 0 fully null rows from uk_affordabil

Checking Null Values for Each Sheet

In [14]:
for sheet_name, df in cleaned_columns.items():
    # Check for missing values per column
    missing_values = df.isnull().sum()

    # Display results
    print(f"\nSheet: {sheet_name}")
    print("Missing Values Per Column:")
    print(missing_values)

    # Check if there are any null values
    total_missing = missing_values.sum()
    if total_missing == 0:
        print("✅ No missing values in this sheet.")
    else:
        print(f"⚠️Total missing values in this sheet: {total_missing}")


Sheet: unemployment_rate
Missing Values Per Column:
Unenployment Rate     0
Population           10
dtype: int64
⚠️Total missing values in this sheet: 10

Sheet: extra_unemployment_rate
Missing Values Per Column:
Unemployment Rate           0
Unemployment (Thousands)    0
    Female                  0
    Male                    0
dtype: int64
✅ No missing values in this sheet.

Sheet: gov_total_expense_revenue
Missing Values Per Column:
Expense               0
Revenue               0
Surplus or Deficit    0
dtype: int64
✅ No missing values in this sheet.

Sheet: money_supply
Missing Values Per Column:
M1                    0
M2                    0
M2 YOY Growth Rate    0
M3                    0
M3 YOY Growth Rate    0
M4                    0
M4 YOY Growth Rate    0
dtype: int64
✅ No missing values in this sheet.

Sheet: bonds
Missing Values Per Column:
5 Year     0
10 Year    0
20 Year    0
dtype: int64
✅ No missing values in this sheet.

Sheet: cpi
Missing Values Per Column:
CPI   

In [15]:
# Function to drop columns with more than 35% missing data with error handling
def drop_columns(df, threshold = 0.35):
    # Calculate the percentage of missing values in each column
    missing_percentage = df.isnull().mean()

    # Identify columns where the percentage of missing values exceeds the threshold
    columns_to_drop = missing_percentage[missing_percentage > threshold].index

    # If ALL columns exceed the threshold, drop only the worst ones and keep at least one column
    if len(columns_to_drop) == len(df.columns):
        print(f"Warning: All columns in the DataFrame exceed the {threshold*100}% missing data threshold.")
        # Keep the column with the least missing data
        column_to_keep = missing_percentage.idxmin()
        columns_to_drop = columns_to_drop.drop(column_to_keep)

    # Drop the identified columns
    return df.drop(columns=columns_to_drop)

cleaned_df = {name: drop_columns(df) for name, df in cleaned_columns.items()}

In [16]:
# Comparison for new dataset
for sheet_name, df in cleaned_df.items():
    print(f"Sheet: {sheet_name}")
    print(f"Columns: {df.columns}")
    print(f"Shape: {df.shape}")
    print(df.head())
    print("\n")

Sheet: unemployment_rate
Columns: Index(['Unenployment Rate', 'Population'], dtype='object')
Shape: (178, 2)
         Unenployment Rate  Population
YYMM                                  
2024/10                4.3         NaN
2024/09                4.3         NaN
2024/08                4.1         NaN
2024/07                4.2         NaN
2024/06                4.2         NaN


Sheet: extra_unemployment_rate
Columns: Index(['Unemployment Rate', 'Unemployment (Thousands)', '    Female',
       '    Male'],
      dtype='object')
Shape: (80, 4)
         Unemployment Rate Unemployment (Thousands)     Female     Male
Date                                                                   
Dec 2024                --                       --         --       --
Nov 2024               4.4                     1568        4.2      4.7
Oct 2024               4.3                     1508          4      4.6
Sep 2024               4.3                     1509        4.1      4.5
Aug 2024         

In [17]:
# Comparison for initial dataset
for sheet_name, df in all_sheets.items():
    print(f"Sheet: {sheet_name}")
    print(f"Columns: {df.columns}")
    print(f"Shape: {df.shape}")
    print(df.head())
    print("\n")

Sheet: unemployment_rate
Columns: Index(['Unenployment Rate', 'Population'], dtype='object')
Shape: (178, 2)
         Unenployment Rate  Population
YYMM                                  
2024/10                4.3         NaN
2024/09                4.3         NaN
2024/08                4.1         NaN
2024/07                4.2         NaN
2024/06                4.2         NaN


Sheet: extra_unemployment_rate
Columns: Index(['Unemployment Rate', 'Unemployment (Thousands)', '    Female',
       '    Male'],
      dtype='object')
Shape: (80, 4)
         Unemployment Rate Unemployment (Thousands)     Female     Male
Date                                                                   
Dec 2024                --                       --         --       --
Nov 2024               4.4                     1568        4.2      4.7
Oct 2024               4.3                     1508          4      4.6
Sep 2024               4.3                     1509        4.1      4.5
Aug 2024         

Aggregated Data for Tables in YYYY/MM Format

In [18]:
# Dictionary to store aggregated annual data
annual_data = {}

for sheet_name, df in cleaned_df.items():
    print(f"\n📄 Processing Sheet: {sheet_name}")

    try:
        # Convert index to string & remove decimal points if read as float
        df.index = df.index.astype(str).str.split(".").str[0]

        # Debug: Show first 5 index values
        print(f"First 5 index values:\n{df.index[:5].tolist()}")

        # Check if all index values match YYYY/MM format
        if not all(re.fullmatch(r"\d{4}/\d{2}", idx) for idx in df.index):
            print(f"⚠️ Skipping {sheet_name}: Index not in 'YYYY/MM' format.")
            continue

        # Extract the year (first 4 characters)
        df["year"] = df.index.str[:4]

        # Group by year and compute the mean for all numeric columns
        annual_dfs = df.groupby("year").mean(numeric_only=True)

        # Store result
        annual_data[sheet_name] = annual_dfs

        # Print the result
        print(annual_dfs.head())

    except Exception as e:
        print(f"⚠️ Error processing {sheet_name}: {e}")


📄 Processing Sheet: unemployment_rate
First 5 index values:
['2024/10', '2024/09', '2024/08', '2024/07', '2024/06']
      Unenployment Rate  Population
year                               
2010           7.875000       62.76
2011           8.058333       63.29
2012           8.016667       63.71
2013           7.633333       64.11
2014           6.300000       64.60

📄 Processing Sheet: extra_unemployment_rate
First 5 index values:
['Dec 2024', 'Nov 2024', 'Oct 2024', 'Sep 2024', 'Aug 2024']
⚠️ Skipping extra_unemployment_rate: Index not in 'YYYY/MM' format.

📄 Processing Sheet: gov_total_expense_revenue
First 5 index values:
['2023/12', '2022/12', '2021/12', '2020/12', '2019/12']
      Expense  Revenue  Surplus or Deficit
year                                      
2010   719.49   570.64             -148.85
2011   723.41   600.04             -123.37
2012   746.45   618.00             -128.45
2013   745.41   649.54              -95.87
2014   767.12   663.27             -103.85

📄 Proces

Now to get all date formats converted to aggregate data